In [1]:
# pip show agentneo

In [2]:
import os
import requests
from dotenv import load_dotenv
import openai
from openai import OpenAI

In [3]:
from agentneo import AgentNeo, Tracer, Evaluation,launch_dashboard
neo_session = AgentNeo(session_name="test")

project_name = "sample"
try:
    neo_session.create_project(project_name=project_name)
except:
    neo_session.connect_project(project_name=project_name)

# Start tracing
tracer = Tracer(session=neo_session)
tracer.start()

INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


Project 'sample' found.
Tracing Started.


In [4]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

@tracer.trace_llm(name="llm_call")
def llm_call(prompt, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content.strip()


llm_call('spell- 10', model='gpt-4')
llm_call('spell- 20', model='gpt-3.5-turbo')
llm_call('spell- 10', model='gpt-4o')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'The spelling of the number 10 is "ten."'

In [5]:
tracer.stop()

Tracing Completed.
Data saved to the database and JSON file.



In [6]:
exe = Evaluation(session=neo_session, trace_id=tracer.trace_id)

In [7]:
# run a single metric
# exe.evaluate(metric_list=['goal_decomposition_efficiency', 
#                          'goal_fulfillment_rate', 
#                          'tool_call_correctness_rate', 
#                          'tool_call_success_rate'])

In [8]:
# #print metric result
# metric_results = exe.get_results()
# metric_results

In [9]:
neo_session.launch_dashboard()

INFO:root:Port 3000 is busy. Finding an available port...
INFO:root:Using port 3012
INFO:root:Dashboard process started successfully
INFO:root:Dashboard launched successfully. Access it at: http://localhost:3012


### Get the trace data

In [10]:
# import requests
# import json

# trace_id = tracer.trace_id 
# port = 2020 # port no of the falsk server
# response = requests.get(f"http://localhost:{port}/api/traces/{trace_id}")

# if response.status_code == 200:
#     trace_data = response.json()
#     print(json.dumps(trace_data, indent=2))
# else:
#     print(f"Error: {response.status_code}")
#     print(response.text)